In [1]:
import pandas as pd 
import numpy as np
from data_preprocessing import combine_datasets
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import torch 
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
import torch.nn as nn
import torchmetrics
import pytorch_lightning as pl
from transformers import DistilBertModel

ImportError: cannot import name 'combine_datasets' from 'data_preprocessing' (/Users/tonderaimadamba/nlp-classification/data_preprocessing.py)

In [ ]:
df = combine_datasets()

In [ ]:
torch.set_default_dtype(torch.float32)

In [ ]:
df = pd.read_csv("CVD_cleaned.csv")

In [ ]:
features = df.loc[:, ~df.columns.isin(["Heart_Disease"])]
targets = df.loc[:,df.columns.isin(["Heart_Disease"])]

In [ ]:


# Here is your dataset
class MyDataset(Dataset):
    def __init__(self, features, targets, train=False,test=False,val=False, transform=None):
        self.train = train
        self.test = test
        self.val = val
        
        self.raw_features = features 
        self.raw_targets = targets
        # Perform one-hot encoding on the features
        self.column_transformer = ColumnTransformer([('one_hot_encoder', 
                                                        OneHotEncoder(sparse=False), 
                                                        self.raw_features.select_dtypes(include=['object']).columns)],
                                                        remainder='passthrough')
        self.new_features = self.column_transformer.fit_transform(self.raw_features)
        self.label_binarizer = LabelEncoder()
        self.new_targets =  self.label_binarizer.fit_transform(self.raw_targets.values)
        
        self._split_data()

        self.targets = self.new_targets[self.indices]   
        self.features = self.new_features[self.indices]
        self.transform = transform 
        
    def _split_data(self):
        idx = list(range(len(self.raw_targets)))
        train_idx, val_test_idx = train_test_split(
                                idx,
                                train_size=0.8,
                                stratify=self.raw_targets,
                                random_state=500)
        
        test_idx,  val_idx, = train_test_split(
                                val_test_idx,
                                train_size=0.8,
                                random_state=500)
        
        # return train_idx, test_idx,  val_idx 

        if self.train:
            self.indices = train_idx
            
        elif self.test:
            self.indices =  test_idx
            
        elif self.val:
            self.indices =  val_idx
        else :
            self.indices = idx
        
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, index):
        sample = self.features[index], self.targets[index]
        if self.transform:
            sample = self.transform(sample)
        return sample

    
class to_tensor:
    def __call__(self,sample):
        features, targets= sample
        return torch.tensor(features,dtype=torch.float32,device='mps:0'), torch.tensor(targets ,dtype=torch.int32,  device='mps:0')
    
    
def transformInput(trainDataset,this_dict):
    return torch.tensor(trainDataset.column_transformer.transform(this_dict),dtype=torch.float32,device='mps:0')
    
#The loader outputs a tensor float64 mac uses mps:0 and float32


In [ ]:

# Here is your PyTorch Lightning model
class LitNetwork(pl.LightningModule):
    def __init__(self, trainDataLoader, testDataLoader, valDataLoader):
        super(LitNetwork, self).__init__()
        self.train_ds = trainDataLoader
        self.test_ds = testDataLoader
        self.val_ds = valDataLoader
        self.linear1 = nn.Linear(48, 25)
        self.linear2 = nn.Linear(25, 2)
        # self.label_binarizer = trainDataLoader.

    def forward(self, x):
        x = torch.sigmoid(self.linear1(x))
        x = self.linear2(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        
        # Calculate accuracy
        preds = torch.argmax(y_hat, dim=1)
        acc = torchmetrics.functional.accuracy(preds,  y)

        # Logging to TensorBoard by default
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return loss
    

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

    def train_dataloader(self):
        return self.train_ds

    def test_dataloader(self):
        return self.test_ds  # return your dataloader
        
    def val_dataloader(self):
        return self.val_ds  # return your dataloader

In [ ]:
def loadModel():
        trainDataset = MyDataset(features, targets,train=True, transform=to_tensor())
        testDataset = MyDataset(features, targets, test=True, transform=to_tensor())
        valDataset = MyDataset(features, targets, val=True, transform=to_tensor())

        trainDataLoader = DataLoader(trainDataset, batch_size=5000, shuffle=True)
        testDataLoader = DataLoader(testDataset, batch_size=5000, shuffle=True)
        valDataLoader = DataLoader(valDataset, batch_size=5000, shuffle=True)

        model = LitNetwork(trainDataLoader,testDataLoader,valDataLoader)
        return model

In [ ]:
trainer = pl.Trainer(max_epochs=1,accelerator="mps")
model = loadModel()
trainer.fit(model)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/tonderaimadamba/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/tonderaimadamba/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/tonderaimadamba/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
bm = {'General_Health' : 'Very Good',
    'Checkup' : 'Within the past year', 
    'Exercise' : 'No', 
    'Skin_Cancer' : 'No', 
    'Other_Cancer' : 'No',
    'Depression' : 'No', 
    'Diabetes' : 'Yes',
    'Arthritis' : 'No', 
    'Sex' : 'Male', 
    'Age_Category' : '70-74',
    'Height_(cm)': 175.0, 
    'Weight_(kg)' : 75.11, 
    'BMI' : 1, 
    'Smoking_History' : 'Yes',
    'Alcohol_Consumption' : 0.0, 
    'Fruit_Consumption' : 30.0,
    'Green_Vegetables_Consumption' : 0.0, 
    'FriedPotato_Consumption' : 4.0}

In [ ]:
def predict(model,df):
    data_df = model.train_ds.dataset.column_transformer.transform(df)
    tensor_df = torch.tensor(data_df,dtype=torch.float32,device='mps:0')
    mps_device = torch.device("mps")
    model.to(mps_device)
    with torch.no_grad():
        pred = model(tensor_df)
    return  nn.functional.softmax(pred)

In [ ]:
trainer.save_checkpoint("example.ckpt")

In [ ]:
new = pd.DataFrame(predict(model,features).cpu().numpy())

/var/folders/8g/g8wt7qjs1d162gzvz0xhgr3c0000gp/T/ipykernel_71092/1454827676.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return  nn.functional.softmax(pred)
